In [ ]:
!git clone https://github.com/ashxjain/YoloV3.git

Cloning into 'YoloV3'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 159 (delta 0), reused 1 (delta 0), pack-reused 156
Receiving objects: 100% (159/159), 12.75 MiB | 9.61 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla K80', major=3, minor=7, total_memory=11441MB, multi_processor_count=13)


In [ ]:
!ls -altrh drive/My\ Drive/EVA/Datasets/YoloV3-PPE/YoloV3_Dataset.zip

-rw------- 1 root root 265M Oct 22 16:16 'drive/My Drive/EVA/Datasets/YoloV3-PPE/YoloV3_Dataset.zip'


### Custom dataset - Setup appropriate files for training

In [ ]:
!unzip drive/My\ Drive/EVA/Datasets/YoloV3-PPE/YoloV3_Dataset.zip

In [ ]:
%%shell
srcFolder="YoloV3_Dataset"
targetDataFolder="YoloV3/data/ppedata"
mkdir -p $targetDataFolder
# create ppe.data file
cat > $targetDataFolder/ppe.data <<EOF
classes=4
train=data/ppedata/ppetrain.txt
valid=data/ppedata/ppetest.txt 
names=data/ppedata/ppe.names
EOF

# copy required files from dataset
cp $srcFolder/classes.txt $targetDataFolder/ppe.names
mkdir -p $targetDataFolder/images && cp $srcFolder/Images/* $targetDataFolder/images/
mkdir -p $targetDataFolder/labels && cp $srcFolder/Labels/* $targetDataFolder/labels/
# Make sure any decimals around 1 i.e 1.xxxxx should be rounded off to 1.0
sed -i 's/1\.[0-9]*/1.0/g'  $targetDataFolder/labels/*

echo "> ppe.data"
cat $targetDataFolder/ppe.data

echo ""; echo "> ppe.names"
cat $targetDataFolder/ppe.names; echo""

> ppe.data
classes=4
train=data/ppedata/ppetrain.txt
valid=data/ppedata/ppetest.txt 
names=data/ppedata/ppe.names

> ppe.names
hardhat
vest
mask
boots


#### Split train/test images (9:1) based of Labels

In [ ]:
import re
import os
import imagesize

srcFolder="YoloV3_Dataset"
targetDataFolder="YoloV3/data/ppedata"
trainFile = open(f'{targetDataFolder}/ppetrain.txt', 'w')
testFile = open(f'{targetDataFolder}/ppetest.txt', 'w')
trainShapesFile = open(f'{targetDataFolder}/ppetrain.shapes', 'w')
testShapesFile = open(f'{targetDataFolder}/ppetest.shapes', 'w')
labelFiles = os.listdir(srcFolder + "/Labels")
imgFiles = os.listdir(srcFolder + "/Images")
count = 0
testCnt = len(labelFiles)/10
trainCnt = len(labelFiles) - testCnt
for file in labelFiles:
  imgParts = file.split(".txt")
  r = re.compile(re.escape(imgParts[0])+".*")
  found = False
  for imgFile in imgFiles:
    if r.match(imgFile):
      shape = imagesize.get(f'{srcFolder}/Images/{imgFile}')
      if count < trainCnt:
        trainFile.write(f'./data/ppedata/images/{imgFile}\n')
        if shape: trainShapesFile.write(f'{shape[0]} {shape[1]}\n')
      else:
        testFile.write(f'./data/ppedata/images/{imgFile}\n')
        if shape: testShapesFile.write(f'{shape[0]} {shape[1]}\n')
      count+=1
      found = True
      break
trainFile.close()
testFile.close()
trainShapesFile.close()
testShapesFile.close()

!echo "Total train images: $(cat YoloV3/data/ppedata/ppetrain.txt | wc -l)"
!echo "Total train image shapes: $(cat YoloV3/data/ppedata/ppetrain.shapes | wc -l)"
!echo "Top 5 lines of train file: $(head -n 5 YoloV3/data/ppedata/ppetrain.txt)"
!echo "";
!echo "Total test images: $(cat YoloV3/data/ppedata/ppetest.txt | wc -l)"
!echo "Total test image shapes: $(cat YoloV3/data/ppedata/ppetest.shapes | wc -l)"
!echo "Top 5 lines of test file: $(head -n 5 YoloV3/data/ppedata/ppetest.txt)"

Total train images: 3174
Total train image shapes: 3174
Top 5 lines of train file: ./data/ppedata/images/img_027.jpg
./data/ppedata/images/8c7d91f6.jpg
./data/ppedata/images/K158.jpg
./data/ppedata/images/Himage_055.jpg
./data/ppedata/images/LImage_002.jpg

Total test images: 347
Total test image shapes: 347
Top 5 lines of test file: ./data/ppedata/images/Fimg_049.jpg
./data/ppedata/images/Mimg_036.jpg
./data/ppedata/images/27581351-painter-worker-wearing-safety-mask-or-safety-work-on-job-painting-of-building-house-or-apartment-wal.jpg
./data/ppedata/images/SImage_75(1).jpg
./data/ppedata/images/young-woman-wearing-medical-face-mask-studio-portrait-picture-id1202943482.jpg


#### Change architecture based on Dataset
* For COCO's 80 classes, YOLOv3's output vector has 255 dimensions ( (4+1+80)*3)* Now we have 4 class, so we would need to change it's architecture.

In [ ]:
%%shell
# Copy the contents of 'yolov3-spp.cfg' file to a new file called 'yolov3-ppe.cfg' file in the data/cfg folder.
cp YoloV3/cfg/yolov3-spp.cfg YoloV3/cfg/yolov3-ppe.cfg

# Search for 'filters=255' (you should get entries entries). Change 255 to *27* = (4+1+4)*3
sed -i 's/filters=255/filters=27/g' YoloV3/cfg/yolov3-ppe.cfg

# Search for 'classes=80' and change all three entries to 'classes=4'
sed -i 's/classes=80/classes=4/g' YoloV3/cfg/yolov3-ppe.cfg

sed -i 's/burn_in.*/burn_in=100/g' YoloV3/cfg/yolov3-ppe.cfg
sed -i 's/max_batches.*/max_batches=5000/g' YoloV3/cfg/yolov3-ppe.cfg
sed -i 's/steps=.*/steps=4000,4500/g' YoloV3/cfg/yolov3-ppe.cfg

# Verify if changes took place successfully
grep "filters=27" YoloV3/cfg/yolov3-ppe.cfg
grep "classes" YoloV3/cfg/yolov3-ppe.cfg
grep "burn_in" YoloV3/cfg/yolov3-ppe.cfg
grep "max_batches" YoloV3/cfg/yolov3-ppe.cfg
grep "steps=" YoloV3/cfg/yolov3-ppe.cfg

filters=27
filters=27
filters=27
classes=4
classes=4
classes=4
burn_in=100
max_batches=5000
steps=4000,4500


#### Download model weights

In [ ]:
# Create a folder called weights in the root (YoloV3) folder
!mkdir -p YoloV3/weights
!cp drive/My\ Drive/weights/*.pt YoloV3/weights/

In [ ]:
!cd YoloV3/; ls weights

best.pt  yolov3_best_300.pt  yolov3-spp-ultralytics.pt
last.pt  yolov3_last_300.pt


In [ ]:
# onnx-tf doesn't work with tf v2.0.0, hence install 2.2.0
#!pip install tensorflow==2.2.0
!pip install onnx
!pip install git+https://github.com/onnx/onnx-tensorflow.git
!pip install tensorflow-addons

     |████████████████████████████████| 7.4MB 6.0MB/s 
  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-ugx0psyx
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-ugx0psyx
  Created wheel for onnx-tf: filename=onnx_tf-1.6.0-cp36-none-any.whl size=199528 sha256=e0706b16bfd0454c714bed4de6007e3319304b85c27e950820f1af846349e3db
  Stored in directory: /tmp/pip-ephem-wheel-cache-nfbysbl4/wheels/54/24/31/8873b33d2d560efdfa1ed6f346df67ef793b1897358705a480
Successfully built onnx-tf


In [ ]:
# Set ONNX_EXPORT global variable to true to get weights in onnx format
!sed -i 's/ONNX_EXPORT = False/ONNX_EXPORT = True/g' YoloV3/models.py
!cd YoloV3; python detect.py --conf-thres 0.1 --cfg cfg/yolov3-ppe.cfg

Namespace(agnostic_nms=False, augment=False, cfg='cfg/yolov3-ppe.cfg', classes=None, conf_thres=0.1, device='', fourcc='mp4v', half=False, img_size=512, iou_thres=0.6, names='data/customdata/custom.names', output='output', save_txt=False, source='data/customdata/images', view_img=False, weights='weights/last.pt')
Using CPU

Fusing layers...
/content/YoloV3/utils/layers.py:60: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if nx == na:  # same shape
graph torch-jit-export (
  %x[FLOAT, 1x3x320x192]
) initializers (
  %465[INT64, 2]
  %466[INT64, 2]
  %module_list.0.0.bias[FLOAT, 32]
  %module_list.0.0.weight[FLOAT, 32x3x3x3]
  %module_list.1.0.bias[FLOAT, 64]
  %module_list.1.0.weight[FLOAT, 64x32x3x3]
  %module_list.10.0.bias[FLOAT, 128]
  %module_list.10.0.weight[FLOAT

In [ ]:
from onnx_tf.backend import prepare
import onnx

TF_PATH = "YoloV3/weights/last.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "YoloV3/weights/last.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

INFO:tensorflow:Assets written to: YoloV3/weights/last.pb/assets


In [ ]:
import tensorflow as tf

TF_PATH = "YoloV3/weights/last.pb" # where the forzen graph is stored
TFLITE_PATH = "YoloV3/weights/last.tflite"
# protobuf needs your virtual environment to be explictly exported in the path
#os.environ["PATH"] = "/opt/miniconda3/envs/convert/bin:/opt/miniconda3/bin:/usr/local/sbin:...."

# make a converter object from the saved tensorflow file
#converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(TF_PATH,  # TensorFlow freezegraph .pb model file
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH,  # TensorFlow freezegraph .pb model file
                                                      )

# tell converter which type of optimization techniques to use
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional
# converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]

converter.experimental_new_converter = True

# I had to explicitly state the ops
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]

tf_lite_model = converter.convert()
# Save the model.
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)

INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


In [ ]:
!cp drive/My\ Drive/weights/last.tflite YoloV3/weights/last.tflite 